# Multi Region IoT - Connect Device to AWS IoT

After the ACM PCA setup has been created and a device certificate has been issued the device is now ready to connect to AWS IoT.  

Because of the JITR configuration for the PCA registered with AWS IoT the device will be provisioned upon the first connection attempt. That means the device certificate will get registered with AWS IoT, an IoT policy will be created if it does not exists, the policy will be attached to the certificate and the certificate will be attached to the device automatically.

The value of the CN in the certificate will be used as thing name.

## Libraries

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient
from os.path import join
import boto3
import json
import logging
import time

#### Note: If you get an error that the AWSIoTPythonSDK is not installed, install the SDK with the command below and import the libraries again!

In [ ]:
!pip install AWSIoTPythonSDK -t .

## Function to Delete a Device
The function deletes a device as well as the attached certificate. It does not delete the policy attached to the certificate.

In [ ]:
def delete_thing(thing_name):
    print("  DELETING {}".format(thing_name))

    try:
        r_principals = c_iot.list_thing_principals(thingName=thing_name)
    except Exception as e:
        print("ERROR listing thing principals: {}".format(e))
        r_principals = {'principals': []}

    #print("r_principals: {}".format(r_principals))
    for arn in r_principals['principals']:
        cert_id = arn.split('/')[1]
        print("  arn: {} cert_id: {}".format(arn, cert_id))

        r_detach_thing = c_iot.detach_thing_principal(thingName=thing_name,principal=arn)
        print("  DETACH THING: {}".format(r_detach_thing))

        r_upd_cert = c_iot.update_certificate(certificateId=cert_id,newStatus='INACTIVE')
        print("  INACTIVE: {}".format(r_upd_cert))

        r_policies = c_iot.list_principal_policies(principal=arn)
        #print("    r_policies: {}".format(r_policies))

        for pol in r_policies['policies']:
            pol_name = pol['policyName']
            print("    pol_name: {}".format(pol_name))
            r_detach_pol = c_iot.detach_policy(policyName=pol_name,target=arn)
            print("    DETACH POL: {}".format(r_detach_pol))

        r_del_cert = c_iot.delete_certificate(certificateId=cert_id,forceDelete=True)
        print("  DEL CERT: {}".format(r_del_cert))

    r_del_thing = c_iot.delete_thing(thingName=thing_name)
    print("  DELETE THING: {}\n".format(r_del_thing))
    

Restore variable that have been defined in the notebook which has been used to create the ACM PCA setup.

In [ ]:
%store -r config
print("config: {}".format(json.dumps(config, indent=4, default=str)))

## IoT Endpoint
To connect the device to AWS IoT we need to get the iot endpoint.

Before the iot endpoint can be determined a boto3 client to AWS IoT must be created.

The device will connect to the master region. After the device has been connected to the master region you can get the endpoint for the slave region and try to connect the device there as well.

In [ ]:
c_iot_m = boto3.client('iot', region_name = config['aws_region_master'])
c_iot_s = boto3.client('iot', region_name = config['aws_region_slave'])

Get the iot endpoint.

In [ ]:
response = c_iot_m.describe_endpoint(endpointType='iot:Data-ATS')

iot_endpoint = response['endpointAddress']
print("iot_endpoint: {}".format(iot_endpoint))

## Connect the Device
Now connect the device to AWS IoT. When a device is reqistered automatically through JITP it will be disconnected automatically after the first connection attempt and the device is being registered. After the connection has timed out the code will wait some seconds and then connect again to AWS IoT. This could take a little bit, so please remain patient.

### Before you start, go to the AWS IoT Console -> Test and subscribe to "$aws/events/#" and "cmd/+/pca"

When a certificate is registered automatically with AWS IoT the service will publish a message to

`$aws/events/certificates/registered/[certificateId]`

If events for "Thing: created, update, deleted" are enbled in your AWS IoT settings AWS IoT will also publish a message to 

`$aws/events/thing/[clientId]/created`

when the device is being created.

Use the same thing name that you used to request a certificate in the previous notebook. Feel free to create more certificates and connect more things.

In [ ]:
thing_name = 'thing-mr04'

root_ca = 'AmazonRootCA1.pem'

device_key_file = '{}.device.key.pem'.format(thing_name)
device_cert_file = '{}.device.cert.pem'.format(thing_name)

# AWS IoT Python SDK needs logging
logger = logging.getLogger("AWSIoTPythonSDK.core")
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)
streamHandler = logging.StreamHandler()
formatter = logging.Formatter("[%(asctime)s - %(levelname)s - %(filename)s:%(lineno)s - %(funcName)s - %(message)s")
streamHandler.setFormatter(formatter)
logger.addHandler(streamHandler)

myAWSIoTMQTTClient = None
myAWSIoTMQTTClient = AWSIoTMQTTClient(thing_name)
myAWSIoTMQTTClient.configureEndpoint(iot_endpoint, 8883)
myAWSIoTMQTTClient.configureCredentials(root_ca, 
                                        join(config['PCA_directory'], device_key_file), 
                                        join(config['PCA_directory'], device_cert_file))

# AWSIoTMQTTClient connection configuration
myAWSIoTMQTTClient.configureAutoReconnectBackoffTime(1, 32, 20)
myAWSIoTMQTTClient.configureOfflinePublishQueueing(-1)  # Infinite offline Publish queueing
myAWSIoTMQTTClient.configureDrainingFrequency(2)  # Draining: 2 Hz
myAWSIoTMQTTClient.configureConnectDisconnectTimeout(10)  # 10 sec
myAWSIoTMQTTClient.configureMQTTOperationTimeout(5)  # 5 sec

# Connect and reconnect to AWS IoT
try:
    myAWSIoTMQTTClient.connect()
except Exception as e:
    logger.error('{}'.format(e))
    time.sleep(5)
    myAWSIoTMQTTClient.connect()

## Verify

Verify that the device has been created in the master and the slave region.

### Master region

In [ ]:
response = c_iot_m.describe_thing(thingName = thing_name)

print("response: {}".format(json.dumps(response, indent=4, default=str)))

### Slave region

In [ ]:
response = c_iot_s.describe_thing(thingName = thing_name)

print("response: {}".format(json.dumps(response, indent=4, default=str)))

## Publish
Publish a message in the master region to verify that the device works as expected.

**You have subsribed to "cmd/+/pca# in the master region?**

In [ ]:
topic = 'cmd/{}/pca'.format(thing_name)
print("topic: {}".format(topic))
message = {"provisioned": "through ACM PCA combined with JITR", "thing_name": "{}".format(thing_name)}

myAWSIoTMQTTClient.publish(topic, json.dumps(message), 0)

Disconnect the device.

In [ ]:
myAWSIoTMQTTClient.disconnect()

Delete the devices. **Note: the IoT policy created through JITP must be deleted manually**

In [ ]:
delete_thing('YOUR_THING_NAME_HERE')